In [191]:
import math

In [265]:
"""atomic unit of micrograd. Basis for building neurons, layers and MLPs."""

class Value:
    
    def __init__(self, data: float, _children:()=None, _op:str=None):
        self.data = float(data)
        self._prev = set(_children) if _children else None
        self.grad = None
        self._op = _op
        self._backward = lambda: None
    
    def __add__(self, other):                
        other = other if isinstance(other, Value) else Value(other)        
        out = Value(self.data + other.data, _children=[self, other], _op='+')
        
        def backward():
            self.grad += out.grad
            other.grad += out.grad
            
        self._backward = backward
        
        return out
    
    def __sub__(self, other):
        other = other if isinstance(other, Value) else Value(other)        
        out = Value(self.data - other.data, _children=[self, other], _op='-')
        
        def backward():
            self.grad += out.grad
            other.grad += -out.grad
        
        self._backward = backward
        
        return out            
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, [self, other], '*')
        
        def backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        
        self._backward = backward
        
        return out
    
    def __pow__(self, power):
        data = self.data**power
        
        def backward():
            self.grad += out.grad * power * self.data**(power-1)
        
        self._backward = backward
        
        return Value(data, None, f'**{power}')
    
    def __truediv__(self, other):        
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data**-1, [self, other], '/')
        
        def backward():
            self.grad += out.grad * (other.data**-1)
            other.grad += out.grad * (-self.data * other.data**-2)
        
        self._backward = backward
        
        return out
    
    def tanh(self,):
        out = Value(math.tanh(self.data), [self], 'tanh')
        
        def backward():
            self.grad = out.grad * (1 - out**2)
            
        self._backward = backward
        
        return out
    
    def relu(self):
        out = Value(0 if self.data < 0 else self.data, [self], 'relu')
        
        def backward():
            self.grad += out.grad * (0 if self.data < 0 else 1.0)
        
        self._backward = backward
        
        return out
    
    def __radd__(self, other):
        return self + other

    def __rsub__(self, other):
        return -1*self + other
    
    def __rmul__(self, other):
        return self * other
    
    def __rtruediv__(self, other):
        return self**-1 * other
        
    def __repr__(self):
        msg = f'Value(data = {str(self.data)}, grad = {str(self.grad)})'
        return msg

In [259]:
a = Value(3)
b = -2

In [260]:
a**b

Value(data = 0.1111111111111111, grad = None)

In [227]:
c._op

'**9'